# Optimization

From the menu above, select "Runtime" -> "Run All".

If you've already run this noteboook recently, click on "Single Task Optimization", then click "Runtime" -> "Run After."

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from summit import *
from multitask import *
import pandas as pd

## Single Task Optimization

Import data

In [3]:
df = pd.read_csv("../data/experiments/cyclopentanol_experiments.csv")

In [4]:
num_experiments = 1

### Generate Suggestions

In [5]:
# Transform data
categorical_columns = ["Solvent", "Ligand"]
for col in categorical_columns:
    df[col] = df[col].str.split(r" \(\d\)", expand=True)[0]
for col in df.columns:
    if "Unnamed" in col:
        df = df.drop(col, axis=1)
df = df.rename(columns={"ResT /min": "ResT", "Temp /°C": "Temp", "Mol% /%": "Mol", "Yield /%": "yld"} )
ds  = DataSet.from_df(df.dropna())
ds

,Type,Solvent,Ligand,ResT,Temp,Mol,yld,SM area,Product area,Biphenyl area,SM conc,Product conc
0,Training,DMA,DPPE,43,94,3,1.28,26148692,349719,1381224,0.979555,0.012732
1,Training,CPME,SPhos,39,188,4,6.13,20486247,1376756,1956733,0.541719,0.03538
2,Training,MeCN,DPPF,87,146,3,1.95,19477539,398988,870757,1.157391,0.023041
3,Training,MeCN,SPhos,114,90,5,1.28,22763771,303134,1277453,0.922024,0.011932
4,Training,THF,DPPF,55,134,1,4.51,11501266,558929,549474,1.083033,0.05115
5,Training,THF,XPhos,94,107,1,1.39,14675342,212567,799457,0.949809,0.01337
6,Training,CPME,XPhos,28,192,1,14.23,8677774,1481940,1687416,0.26609,0.044162
7,Training,CPME,XPhos,82,117,3,2.12,13591653,303444,785593,0.895195,0.019423
8,Training,DMA,DPPE,58,209,2,69.98,6036822,14481792,239634,1.303476,3.038868
9,Training,MeCN,XPhos,116,120,2,1.54,8335694,134341,449286,0.959979,0.015036


In [6]:
# Create domain
domain = Domain()

# Solvents: CPME, DMA, MeCN, THF, Chloroform
# Ligand: DPPF, Sphos, Xphos, DPEPhos, DPPE
# ResT: 20 - 120 mins
# Temp: 80 - 210 deg
# Mol%: 1 - 5%
domain += CategoricalVariable(
    "Solvent",
    "Solvent used for the reaction", 
    levels=["CPME", "DMA", "MeCN", "THF", "Chloroform"]
)
domain += CategoricalVariable(
    "Ligand",
    "Ligand used for the reaction",
    levels=["DPPF", "SPhos", "XPhos", "DPEPhos", "DPPE"]
)
domain += ContinuousVariable(
    "ResT",
    "Residence Time",
    bounds=(20, 120)
)
domain += ContinuousVariable(
    "Temp",
    "Reaction temperature in deg C",
    bounds=(80,210)
)
domain += ContinuousVariable(
    "Mol",
    "Catalyst mol percent",
    bounds=(1,5)
)
domain += ContinuousVariable(
    "yld",
    "Reaction yield",
    bounds=(0, 100),
    is_objective=True,
    maximize=True
)
print("Domain")
domain

Domain


Name,Type,Description,Values
Solvent,"categorical, input",Solvent used for the reaction,5 levels
Ligand,"categorical, input",Ligand used for the reaction,5 levels
ResT,"continuous, input",Residence Time,"[20,120]"
Temp,"continuous, input",Reaction temperature in deg C,"[80,210]"
Mol,"continuous, input",Catalyst mol percent,"[1,5]"
yld,"continuous, maximize objective",Reaction yield,"[0,100]"


In [7]:
cat_mappings = {}
cat_dimensions = []
for i, v in enumerate(domain.input_variables):
    if v.variable_type == "categorical":
        cat_mapping = {l: i for i, l in enumerate(v.levels)}
        cat_mappings[v.name] = cat_mapping
        cat_dimensions.append(i)


In [8]:
combos = domain.get_categorical_combinations()
for v in domain.input_variables:
    if v.variable_type == "categorical":
        combos[v.name] = combos[v.name].replace(cat_mappings[v.name])

In [9]:
strategy = NewSTBO(domain, categorical_method=None, brute_force_categorical=True)
suggestions = strategy.suggest_experiments(int(num_experiments), prev_res=ds)
suggestions = suggestions.round(0)
suggestions

/Users/Kobi/Documents/Research/phd_code/multitask/multitask/mt.py:735: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  return torch.tensor(bounds).T.float()


> /Users/Kobi/Documents/Research/phd_code/multitask/multitask/mt.py(673)suggest_experiments()
    672         ipdb.set_trace()
--> 673         if self.categorical_method is None:
    674             for i, v in enumerate(self.domain.input_variables):



ipdb>  c


,Solvent,Ligand,ResT,Temp,Mol,strategy
0,CPME,DPPF,120.0,210.0,1.0,STBO


### Download suggestions